# Introduction

In this notebook, we will create a simple multi-agent system using OpenAI's swarm library. The system will consist of multiple specialized agents that can perform tasks such as greeting the user, telling jokes, fetching news headlines, accessing calendar events, and providing weather updates. A main agent will orchestrate these specialized agents by delegating user requests to the appropriate agent based on the content of the request.



# Setup

https://github.com/openai/swarm

First, we need to set up our environment by installing the required packages and setting up API keys.

In [ ]:
# Check the Python version to ensure compatibility
import sys
print(sys.version)

3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]


This will output the Python version we are using, which should be compatible with the required libraries (Python 3.10 or higher is needed).



In [ ]:
# Install the 'swarm' library from GitHub and the 'requests' library for HTTP requests
!pip install git+https://github.com/openai/swarm.git
!pip install requests

  Cloning https://github.com/openai/swarm.git to /tmp/pip-req-build-r0aqm8hi
  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git /tmp/pip-req-build-r0aqm8hi
  Resolved https://github.com/openai/swarm.git to commit 9db581cecaacea0d46a933d6453c312b034dbf47
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

* swarm: A library for creating and managing multiple AI agents.
* requests: A library for making HTTP requests, which we'll use to interact with APIs.

In [ ]:
# Import the 'userdata' module to securely handle user data (API keys)
from google.colab import userdata

# Retrieve the OpenAI API key and News API key from user data
openai_api_key = userdata.get('genai_course')
news_api_key = userdata.get('news_api')

Note: The google.colab.userdata module is specific to Google Colab and allows you to securely store and retrieve user data like API keys without hardcoding them in your code.

In [ ]:
# Import required libraries
import os
from swarm import Swarm, Agent
import requests
from IPython.display import display, Markdown

In [ ]:
# Set the API keys as environment variables
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['NEWS_API_KEY'] = news_api_key

We set our API keys as environment variables so that they can be accessed securely by the agents.



# Agent 1 - Greeting Agent

We will define our first agent, the Greeting Agent, which will greet the user and tell a joke.



In [ ]:
# Define the Greeting Agent
greeting_agent = Agent(
    name="Greeting Agent",
    instructions="You are a friendly assistant that greets the user and tells a joke.",
)

* **Name**: Identifies the agent.
* **Instructions**: Provides the agent with context on how to respond.

# Agent 2 - News Agent

Next, we define the News Agent, which fetches the latest news headlines.

In [ ]:
def fetch_news(country="us", category=None):
    """
    Fetches the top news headlines for a given country and category.

    Args:
        country (str): The country code (default is "us").
        category (str): The news category (optional).

    Returns:
        str: A string containing the top news headlines or an error message.
    """
    base_url = "https://newsapi.org/v2/top-headlines"
    params = {
        "country": country,
        "apiKey": news_api_key
    }

    # Add a category if provided
    if category:
        params["category"] = category

    # Make a GET request to the News API
    response = requests.get(base_url, params=params)
    data = response.json()

    # Check if the request was successful
    if response.status_code == 200 and 'articles' in data:
        # Extract the titles of the first 10 articles
        headlines = [article['title'] for article in data['articles'][:10]]
        return f"Here are the top news for {country.upper()} ({category or 'all categories'}):\n" + "\n".join(headlines)
    else:
        # Return an error message if the request failed
        return f"Sorry, I couldn't fetch the news at the moment: {data.get('message', 'Unknown error')}"


This function uses the News API to fetch the latest headlines based on the specified country and category.

In [ ]:
# Define the News Agent
news_agent = Agent(
    name="News Agent",
    instructions="You provide the top news headlines for a given country and category.",
    functions=[fetch_news]
)

* **Functions**: List of functions that the agent can use to perform tasks.

# Orchestrating the agents

We define how the main agent will delegate tasks to specialized agents.

In [ ]:
# Define function that transfers the task to the agent
def transfer_to_agent(agent_name):
  agents = {
      "Greeting Agent": greeting_agent,
      "News Agent": news_agent
  }
  return agents[agent_name]

* **Functionality**: Returns the agent instance based on the provided agent name.
* **Usage**: The main agent will use this function to transfer control to the appropriate agent.

In [ ]:
# Define the Main AI Agent
main_agent = Agent(
    name = "Main Agent",
    instructions = """
    You are the main assistant.
    BAsed on the user's request, you decide which specialized agent should handle the task.
    - If the user wants a greeting or a joke, transfer to the Greeting Agent.
    - If the user wants to fetch the news, transfer to the News Agent.
    """,
    functions = [transfer_to_agent]
)

* **Instructions**: Guides the main agent on how to delegate tasks.
* **Functions**: Includes the transfer_to_agent function to enable delegation.

# Start the Connection to the OpenAI API


We initialize the Swarm client, which handles communication with the OpenAI API and manages the agents.

In [ ]:
# Start the connection the OpenAI API
client = Swarm()

We simulate a user interaction with the main agent.

In [ ]:
# Run the application
response = client.run(
    agent = main_agent,
    messages = [{"role": "user",
                 "content": "good morning"}]
)
print(display(Markdown(response.messages[-1]['content'])))

Good morning! I hope you're having a fantastic start to your day. Want to hear a joke to get those morning smiles going?

None


* **Agent**: We specify the main_agent as the agent to handle the interaction.
* **Messages**: We pass a list of messages, starting with the user's input "good morning".
* **Response Handling**: We display the assistant's response in Markdown format for better readability.

In [ ]:
response.agent

Agent(name='Greeting Agent', model='gpt-4o', instructions='You are a friendly assistant that greets the user and tells a joke.', functions=[], tool_choice=None, parallel_tool_calls=True)

This line outputs the agent that provided the final response, which helps us verify which specialized agent handled the request.

# Homework


* **Access Tasks and Calendar for the Day**: Extend the application by creating a new agent that can retrieve and display your tasks and calendar events for the day. You might need to integrate with a calendar API like Google Calendar.

* **Weather for the Day**: Create an agent that provides the weather forecast for the day. You can use a weather API like OpenWeatherMap. Remember to handle API keys and context variables appropriately.

Feel free to experiment and enhance the functionality of the agents to make them more interactive and user-friendly.